## Now I got the data for the results with Tensorflow dence layers. I categorized good MAPE result country and not good one. 

Now I will train category 2 with linear regression to get better MAPE results. 

### Things to do:

* Load result data with category. 
* Load train data. 
* Grab the cfips on category 2.
* train with linear regression if the country incategory 2, then update MAPE. 


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from helper_functions import *

In [51]:
data = pd.read_csv("result_dense_with_category.csv")
train = pd.read_csv("train.csv")

In [52]:
# Grab the cfips on category 2
cfips_category1 = data[data['category'] == 1]['Country'].unique()

In [53]:
len(cfips_category1)

609

In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3135 entries, 0 to 3134
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Country   3135 non-null   int64  
 1   MAPE      3135 non-null   float64
 2   Density   3135 non-null   float64
 3   Active    3135 non-null   int64  
 4   forecast  3135 non-null   object 
 5   category  3135 non-null   int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 147.1+ KB


### Work on this for loop to get new MAPE with new training. Let's see if the total MAPE increase. 

Code above is the Renear regression training. 

In [43]:
np.arange(31,39).reshape((-1,1))

array([[31],
       [32],
       [33],
       [34],
       [35],
       [36],
       [37],
       [38]])

In [65]:
THRESHOLD = 0.103

IDS = train.cfips.unique()
x_train = np.arange(31).reshape((-1,1))
x_test = np.arange(31,39).reshape((-1,1))

# Based on the Chris's rule, SMAPE > THRESHOLD -> Use linear regrssion 
lr_or_non_lr = []  # If use linear regression, label as 1, if not label as 0
smape = []
mape = []
predict = []
new_predict = []
new_category = []

for cfips in IDS:
    c = cfips


    # If the cfips in the category 2, then train with linear regression and update MAPE
    if c in cfips_category1:
        category_new = 'tf_dense'
        new_category.append(category_new)
        mape.append(np.float64(data[data['Country'] == c]["MAPE"]))
        new_predict.append(data[data['Country'] == c]['forecast'])
    # If category 1, keep the MAPE score
    else:
        df = train.loc[train['cfips'] == c]
        last = df['microbusiness_density'].values[-9]

        # Fit linear regression
        model = LinearRegression()
        model.fit(x_train,df['microbusiness_density'][:31])
        p = model.predict(x_train)

        # Compute train error
        err = p - df['microbusiness_density'][:31].values
        rng = df['microbusiness_density'].max() - df['microbusiness_density'].min()
        #print(f'Error: {err}\nRange: {rng}')

        # Determin if time series is linear or not
        s = 0
        for k in range(31):
            e = np.abs(err[k])
            #print(f'e : {e}')
            r = e / rng # absolute error divided by range
            #print(f'r: {r}')
            s += r
        #mape.append(s) # Grab MAPE to compare with other data
        #print(f's: {s}')
        s = s/39 # now S is MAPE mean absolute percentage error
        smape.append(s) 
        #print(f'Divide s by the number of data -> MAPE: {s}')

        # Infer test data with linear regression
        p2 = model.predict(x_test)
        shift = last - p2[0]
        if s <THRESHOLD: 
            category_new = 'LR'
            new_category.append(category_new)
            preds = p2[:]+shift
            label = 0
            lr_or_non_lr.append(label)
            test_label = np.expand_dims(np.array(train[train['cfips'] == c]['microbusiness_density'][31:39]), axis=0)
            results = evaluate_preds(test_label, preds)
            m = results['mape']
            predict.append(np.array(preds).reshape((-1)))
            mape.append(m)
            new_predict.append(preds)
        else: 
            category_new = 'Same value'
            new_category.append(category_new)
            preds = [last]*8
            label = 1
            lr_or_non_lr.append(label)
            test_label = np.expand_dims(np.array(train[train['cfips'] == c]['microbusiness_density'][31:39]), axis=0)
            results = evaluate_preds(test_label, preds)
            m = results['mape']
            predict.append(np.array(preds).reshape((-1)))
            mape.append(m)
            new_predict.append(preds)

count_lr = lr_or_non_lr.count(0)
count_non_lr = lr_or_non_lr.count(1)
print(f'The data predict with Linear Regression: {count_lr}')
print(f'The data not predict with Linear Regression: {count_non_lr}')
print(f'Model: {model}\nMean SMAPE: {np.mean(smape)}\nMin SMAPE: {np.min(smape)}\nMax SMAPE: {np.max(smape)}')
                
        
data['new_MAPE'] = mape
data['new_predict'] = new_predict
data['new_category'] = new_category
data[['MAPE', 'new_MAPE']]

The data predict with Linear Regression: 1023
The data not predict with Linear Regression: 1503
Model: LinearRegression()
Mean SMAPE: 0.11579484807089503
Min SMAPE: 0.0018782232910424662
Max SMAPE: 0.2993328663980672


,MAPE,new_MAPE
0,1.504614,0.992271
1,4.143341,5.230218
2,1.395184,1.395184
3,5.228748,3.574082
4,1.442244,1.442244
...,...,...
3130,8.937141,2.450784
3131,3.437833,1.586990
3132,4.339747,1.381605
3133,2.373714,3.608545


#### Check how many MAPE incresed

In [56]:
print(f"Old Mean MAPE: {data['MAPE'].mean()}\nNew Mean MAPE: {data['new_MAPE'].mean()}")

Old Mean MAPE: 27.99670683904312
New Mean MAPE: 8926.5638727907


In [57]:
increased = 0
unchanged = 0
decreased = 0
for i in range(len(data)):
    if data['MAPE'].iloc[i] > data['new_MAPE'].iloc[i]:
        increased += 1
    elif data['MAPE'].iloc[i] < data['new_MAPE'].iloc[i]:
        decreased += 1
    else:
        unchanged += 1
print(f"MAPE Increased: {increased}\nMAPE Decreased: {decreased}\nUnchanged: {unchanged}")

MAPE Increased: 1804
MAPE Decreased: 722
Unchanged: 609


## Conclusion:

MAPE Increased most of them by using Linear Regression with the SMAPE < 1.03, and use the last microbusiness density value for the SMAPE larger than 1.03 with the Country I categorised 2 after I trained on tensorflow Dense layer result. However, the mean values decreased big number and I think it is because few of them decreased MAPE a lots. I will find out which one got worse and I will apply different method on those bad one. And I will add the one more step that to compare with old MAPE and pick the better one. For now, I will use this model and submit to see if this model get better result than old one with just tf dense model. 




## Next step:

* Compare old MAPE and new MAPE then if it increase (1905 of them increased), updatte to new MAPE, if decrease (621 of them), then use old one stead. 
* Those 621 of them, I will apply diffrent model to get the result. 
* I will train with entire data with each method, then create the result table, so I can compare which one is the best. 

In [67]:
data.to_csv("result_after_LR.csv", index=False)

In [68]:
d = pd.read_csv("result_after_LR.csv")

In [69]:
d.head()

,Country,MAPE,Density,Active,forecast,category,new_MAPE,new_predict,new_category
0,1001,1.504614,3.463856,1472,[3.3091352 3.31727028 3.32801533 3.33693123 3...,2,0.992271,[3.3344314 3.34608375 3.35773609 3.36938844 3...,LR
1,1003,4.143341,8.359798,14320,[7.85833788 7.88518858 7.90744925 7.90828514 7...,2,5.230218,"[7.8232999, 7.8232999, 7.8232999, 7.8232999, 7...",Same value
2,1005,1.395184,1.232074,244,[1.16141069 1.16513038 1.17019248 1.18426347 1...,1,1.395184,2 [1.16141069 1.16513038 1.17019248 1.18426...,tf_dense
3,1007,5.228748,1.287240,229,[1.22299695 1.22010887 1.22093809 1.21361172 1...,2,3.574082,"[1.2366498, 1.2366498, 1.2366498, 1.2366498, 1...",Same value
4,1009,1.442244,1.831783,813,[1.75237012 1.76360607 1.77065146 1.7765491 1...,1,1.442244,4 [1.75237012 1.76360607 1.77065146 1.77654...,tf_dense


## Next step

* Create the list of Country by new_category then predict with what category says.

In [79]:
cat_1 = d[d['new_category'] == 'tf_dense']['Country'].unique()
cat_2 = d[d['new_category'] == 'LR']['Country'].unique()
cat_3 = d[d['new_category'] == 'Same value']['Country'].unique()

In [80]:
len(cat_1), len(cat_2), len(cat_3)

(609, 1023, 1503)

## Sudo code for next step:

for loop for cfips:

if cfips in cat_1:

    run tensorflow dense and get prediction. 
    predict result to numpy array
    then add to the list of forcasts
    
if cfips in cat_2:

    run Linear regression and get prediction
    add to the list of forecasts
    
if cfips in cat_3:

    last number will be theprediction.
    add to the list of forecasts
    
convert to numpy array - np.array(forecasts).reshape((-1)) to convert 1D array, so I can submit the result.  